<center>
    
## Progetto: Foundation of Computer Science

### Università degli studi di MIlano-Bicocca

### Corso di laurea in Data Science

### P. Bonardi & L. Ravazzi
    
</center>

You have to work on the University dataset. It contains three different university rankings:

1. The Times Higher Education World University Ranking, shortly Times,
2. the Academic Ranking of World Universities, shortly Shanghai,
3. the Center for World University Rankings, shortly cwur.

### IMPORT DATA

In [4]:
import pandas as pd
import numpy as np
import re

Descrizione del dataset **_Times Higher Education World University Ranking_** e dei suoi attributi:

1. **world_rank**: rank mondiale delle università rispetto all'anno definito.
2. **university_name**: nome dell'università.
3. **country**: stato di appartenenza.
4. **teaching**:  punteggio legato alle attività di apprendimento (rapporto tra dottorati e laureati, sondaggi di repitazione, rapporto staff/studenti etc.).
5. **international**: score legato al personale e studenti stranieri.
6. **research**: score riferito al "Reputation survey", reddito e produttività della ricerca accademica.
7. **citations**
8. **income**
9. **total_score**: punteggio dal quale si estrae il ranking.
10. **num_students**: numero di studenti.
11. **student_staff_ratio**: rapporto tra studenti e staff. 
12. **international_students**: percentuale di studenti stranieri.
13. **female_male_ratio**: numero di femmine vs. numero di maschi. 
14. **year**: anno di riferimento per tutti i dati ed in particolare, per il ranking.

https://www.timeshighereducation.com/world-university-rankings/world-university-rankings-2021-methodology

Ogni ranking considera un tot numero di università all'anno che possono cambiare nel corso del tempo. Per ogni anno, viene definito un score totale dal quale si ricava il rank. Ogni riga del dataset si riferisce alle caratteristiche di una università in quell'anno quindi i valori potrebbero modificarsi, a priori, modificarsi nel tempo. Tuttavia, si osserva che alcuni campi, come il numero di studenti o il rapporto femmine/maschi, rimangono invariati durante gli anni.

In [24]:
  
            #-- Times Higher Education World University Ranking
path_pietro = "/Users/pietrobonardi/GitHubSpace/FoCS/dataset/"
ts = pd.read_csv(path_pietro+'timesData.csv')
ts.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


In [25]:
ts.tail()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,"21,958",15.3,3%,48 : 52,2016
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,"31,268",28.7,2%,36 : 64,2016
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,"4,122",3.7,3%,NaN,2016
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,"10,117",12.1,8%,28 : 72,2016
2602,601-800,Yuan Ze University,Taiwan,16.2,17.7,18.3,28.6,39.8,-,"8,663",20.6,4%,43 : 57,2016


In [26]:
ts.shape

(2603, 14)

In [27]:
ts.columns

Index(['world_rank', 'university_name', 'country', 'teaching', 'international',
       'research', 'citations', 'income', 'total_score', 'num_students',
       'student_staff_ratio', 'international_students', 'female_male_ratio',
       'year'],
      dtype='object')

In [28]:
# Quante università sono state considerate per ogni anno?
# Questo raggruppamento ha senso solo perché in ogni anno,
# ogni università si presenta solo una volta nel ranking.
ts.groupby(['year'])['university_name'].count()
# Nel tempo sono aumenate: prestare attenzione al
# fatto che la stessa università può presentarsi diverse volte negli
# anni oppure sparire. 

year
2011    200
2012    402
2013    400
2014    400
2015    401
2016    800
Name: university_name, dtype: int64

In [29]:
ts.isnull().any()

world_rank                False
university_name           False
country                   False
teaching                  False
international             False
research                  False
citations                 False
income                    False
total_score               False
num_students               True
student_staff_ratio        True
international_students     True
female_male_ratio          True
year                      False
dtype: bool

In [30]:
# Prendo solo gli attributi che mi servono.
ts = ts[['world_rank', 'university_name', 'country', 'income', 'num_students', 'female_male_ratio', 'year']]

In [31]:
# controllo se vi sono delle righe uguali.
ts[ts.duplicated()]

,world_rank,university_name,country,income,num_students,female_male_ratio,year


Descrizione del dataset **_Center for World University Rankings_** e dei suoi attributi:

1. **world_rank**: rank globale.
2. **institution**: nome dell'università.
3. **country**: stato.
4. **national_rank**: rank nazionale.
5. **quality_of_education**: score legato al numero di studenti che hanno vinto "premi accademici" rispetto alle dimensioni dell'università.
6. **alumni_employment**: score legato al numero di studenti che ricoprono posizioni ambiziose in aziende mondiali di spicco relativamente alla dimensione dell'università.
7. **quality_of_faculty**: misurata rispetto al numero di membri della facolatà che ha vinto il maggior numero di premi.
8. **publications**: misurata rispetto al numero di articoli presenti nelle "top-tier journals".
9. **influence**: misurata rispetto al numero di articoli presenti in riviste di spicco.
10. **citations**: citazioni
11. **broad_impact**: internazionalizzazione.
12. **patents**: brevetti.
13. **score**: punteggio che permette di definire il ranking.
14. **year**: anno di riferimento dei dati. 

In [32]:

                #-- Center for World University Ranking.

cwur = pd.read_csv(path_pietro+'cwurData.csv')
#cwur = pd.read_csv('Dataset/cwurData.csv')
cwur.rename({'institution': 'university_name'}, inplace = True, axis = 1)
cwur.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [33]:
cwur.tail()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,998,Federal University of Ceará,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,999,University of A Coruña,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015
2199,1000,China Pharmaceutical University,China,83,367,567,218,861,991,812,981.0,547,44.02,2015


In [34]:
cwur.shape

(2200, 14)

In [35]:
cwur.columns

Index(['world_rank', 'university_name', 'country', 'national_rank',
       'quality_of_education', 'alumni_employment', 'quality_of_faculty',
       'publications', 'influence', 'citations', 'broad_impact', 'patents',
       'score', 'year'],
      dtype='object')

In [36]:
cwur.groupby(['year'])['university_name'].count()

year
2012     100
2013     100
2014    1000
2015    1000
Name: university_name, dtype: int64

In [37]:
cwur.isnull().any()

world_rank              False
university_name         False
country                 False
national_rank           False
quality_of_education    False
alumni_employment       False
quality_of_faculty      False
publications            False
influence               False
citations               False
broad_impact             True
patents                 False
score                   False
year                    False
dtype: bool

In [38]:
# seleziono solo gli attributi che mi servono.
cwur = cwur[['university_name', 'country', 'world_rank', 'year']]

In [39]:
# il rank nazionale non mi porta dei doppioni. 
cwur[cwur.duplicated()]

,university_name,country,world_rank,year


Descrizione del dataset **_Academic Ranking of World Universities_** e dei suoi attributi:
1. **world_rank**: rank mondiale.
2. **university_name**: nome dell'università.
3. **national_rank**: rank nazionale.
4. **total_score**: score dal quale si ricava il rank.
5. **alumni**: percentuale di studenti che hanno ottenuto il nobel o la medaglia Field.
6. **award**: percentuale di staff che ha vinto il nobel in alcuni campi e la medaglia Field.
7. **hici**: oercentuale di Highly Cited Researchers selezionate.
8. **ns**: numero di articoli pubbicati nel campo Nature and Science tra il 2015 e 2019.
9. **pub**: indicatore relativo alle pubblicazioni.
10. **pcp**: punteggio pesato dei precedenti cinque indicatori.
11. **year**: anno di riferimento.

In [40]:

            #-- Academic Ranking of World Universities.
shang = pd.read_csv(path_pietro+'shanghaiData.csv')
#shang = pd.read_csv('Dataset/shanghaiData.csv')
shang.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


In [41]:
shang.tail()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
4892,401-500,University of Trieste,11-20,NaN,0.0,0.0,5.0,10.9,25.1,20.1,2015
4893,401-500,University of Zaragoza,9-13,NaN,0.0,0.0,7.6,5.1,33.3,13.1,2015
4894,401-500,Utah State University,126-146,NaN,13.6,0.0,3.6,10.8,25.1,15.5,2015
4895,401-500,Vienna University of Technology,4-6,NaN,0.0,0.0,0.0,12.2,28.8,22.9,2015
4896,401-500,Wake Forest University,126-146,NaN,0.0,0.0,14.9,7.5,25.0,11.9,2015


In [42]:
shang.shape

(4897, 11)

In [43]:
shang.columns

Index(['world_rank', 'university_name', 'national_rank', 'total_score',
       'alumni', 'award', 'hici', 'ns', 'pub', 'pcp', 'year'],
      dtype='object')

In [44]:
shang.groupby(['year'])['university_name'].count()

year
2005    500
2006    500
2007    510
2008    503
2009    501
2010    500
2011    500
2012    284
2013     98
2014    500
2015    500
Name: university_name, dtype: int64

In [45]:
shang.isnull().any()

world_rank         False
university_name     True
national_rank       True
total_score         True
alumni              True
award               True
hici                True
ns                  True
pub                 True
pcp                 True
year               False
dtype: bool

In [46]:
# non posso permettermi di avere università senza nome.
shang = shang[shang.university_name.notnull()]

In [47]:
# seleziono solo gli attributi che mi servono.
shang = shang[['university_name', 'world_rank', 'year']]

In [48]:
shang[shang.duplicated()]

,university_name,world_rank,year
1234,Queen's University,203-304,2007
1740,Queen's University,201-302,2008
2742,Queen's University,201-300,2010


In [49]:
shang[shang['university_name'] == 'Queen\'s University']

,university_name,world_rank,year
164,Queen's University,153-202,2005
660,Queen's University,151-200,2006
1233,Queen's University,203-304,2007
1234,Queen's University,203-304,2007
1739,Queen's University,201-302,2008
1740,Queen's University,201-302,2008
2240,Queen's University,201-302,2009
2741,Queen's University,201-300,2010
2742,Queen's University,201-300,2010
3240,Queen's University,201-300,2011


In [50]:
# l'università del Queen si ripete perché il rank nazionale cambia ma 
# quello mondiale rimane costante. Oer i nostri scopi possiamo eliminarle.
shang = shang[~shang[['university_name', 'year', 'world_rank']].duplicated()]
# shang[shang.university_name.str.contains('^Queen\'s University$')].sort_values('year')

### 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes.

**_Strategia_**: 

1. **Due università potrebbero avere lo stesso nome.** Gli attributi che ci servono sono il nome dell'università e l'anno di riferimento. Se due università avessero lo stesso nome, potrebbe essere un problema. Questo non si verifica all'interno dello stesso anno ma potrebbe verificarsi prendendo tutto il dataset. Analizziamo questo secondo caso. Se due università avessero lo stesso nome e risiedessero nello stesso stato, non avremmmo gli elementi per poterli discriminare, mentre se lo stato fosse diverso e il nome lo stesso, potremmo distinguerli. In realtà, questo secondo aspetto accade per sole due università ma a causa di un errore di battitura: infatti, le università di Aston University e University of Hawai’i at Mānoa sono presenti in due stati diversi in tutto il dataset ma perché il secondo si trova, oltre che in Unisted States of America, anche in United States of America, idem per l'altro ma con UK. Ho corretto il problema: possiamo quindi dire che il nome dell'università è unico per ognuna di esse nel dataset (a meno nel caso in cui due università abbiano lo stesso nome nello stesso stato). Nel dataset così creato esiste quindi una _corrispondenza biunivoca tra il nome e l'università_ .

2. **Cosa fare con le università che si presentano una sola volta**. Queste università non hanno una data più recente e una meno recente quindi ci sono quattro possibili opzioni di inserimento: dataframe dei recenti, dataframe dei meno recenti, vengono distribuite secondo un criterio oppure non si considerano. Io scelgo di percorrere l'ultima strada perché le richieste successive chiedono di calcolare delle differenze che presuppongono l'esistenza dei dati in diversi anni.

In [352]:

# -- Controllo del nome delle università.

# queste sono le università che nel tempo sono state valutate dal Times. 
# non ci sono valori nulli e nemmeno valori nulli scritti in modo diverso da np.nan o None.
len(set(ts.university_name)), ts[['university_name', 'year']].isnull().any()

(818,
 university_name    False
 year               False
 dtype: bool)

In [353]:
def check_name(df, which_1 = True, which_2 = True):
    if which_1:
        print('Nello stesso anno, sono presenti università con lo stesso nome?')
        check = df.groupby(['year', 'university_name'], as_index = False).size()
        if len(check[check > 1]) == 0:
            print('No')
        else: 
            print('Yes')
    if which_2:
        print('Esistono università con lo stesso nome ma presenti in stati diversi?')
        check = df.groupby(['university_name'])['country'].nunique()
        appo = pd.merge(df[['university_name','country']], check[check > 1].reset_index()[['university_name']], on = 'university_name').drop_duplicates()
        if appo.shape[0] != 0:
            print(appo)
        else:
            print('No')

In [354]:
check_name(ts)

Nello stesso anno, sono presenti università con lo stesso nome?
No
Esistono università con lo stesso nome ma presenti in stati diversi?
No


In [355]:
# all'interno dello stesso anno, le università hanno tutti nomi
# diversi quindi non vi sono università con lo stesso nome. 

ts['country'] = ts['country'].str.replace('Unisted States of America', 'United States of America', regex = True)
ts['country'] = ts['country'].str.replace('Unted Kingdom', 'United Kingdom', regex = True)


In [356]:
check_name(ts)

Nello stesso anno, sono presenti università con lo stesso nome?
No
Esistono università con lo stesso nome ma presenti in stati diversi?
No


In [357]:
def select_old_and_new(df):
    df_size = df.groupby('university_name').size()
    # non voglio il nome come indice.
    df_size = pd.DataFrame(df_size).reset_index()
    df_size.columns = ['university_name','Size']

    mrg = pd.merge(df, df_size, how = 'inner', on = 'university_name')
    mrg = mrg[mrg['Size'] != 1]
    
    # lavoro sulla loc perché la tabella merge ha un indice esplicito e uno implicito di default.
    # poiché mrg è un subset di ts, allora l'indice esplicito di mrg è parte di quello implicito di ts.
    # Quindi durante la groupby, io ottengo un indice esplicito di mrg che vorrà la loc. 
    # la idxmax e idxmin resituiscono l'indice esplicito.
    
    # testare con l'altra in cui si sorta (pietronotebook)
    df_old = mrg.loc[mrg.groupby(['university_name'])['year'].idxmin(),:].sort_values('year')
    df_new = mrg.loc[mrg.groupby(['university_name'])['year'].idxmax(),:].sort_values('year')
    
    sing = sum(df_size.Size == 1)
    print(f"Università singleton: {sing}, Università non singleton: {df_size.shape[0] - sing}")
    print(f"Dimensione df recenti {df_new.shape} e meno recenti {df_old.shape}")
    return df_old, df_new


In [358]:

#-- Problema dela presenza una sola volta di alcune università.

ts_old, ts_new = select_old_and_new(ts)

Università singleton: 344, Università non singleton: 474
Dimensione df recenti (474, 8) e meno recenti (474, 8)


###  2. For each university, compute the improvement in income between the least recent and the most recent data points

**_Strategia_**: unisco i dataframe ottenuti precedentemente e calcolo la differenza tra i redditi. L'indice della join sarà il nome dell'università per la corrispondenza biunivoca ottenuta ed inoltre, la join sarà di tipo _inner_ perché per ogni università voglio recuperare entrambi gli anni. Il problema principale è che vi sono alcuni valori - nei redditi che saranno eliminati perché non permettono il calcolo della differenza.

_IMPORTANTE_ :
questo risultato non tiene in considerazione tutte quelle università che non hanno uno dei due income (cioè non posso calcolare una differenza) e che sono presenti solo una volta nel dataframe ts. Infatti, per quest'ultimo caso, la differenza di reddito sarebbe sempre zero ma non perché non c'è stata variazione nel tempo (che potrebbe accadere) ma solo perché sto considerando lo stesso valore. Quindi il risultato di variazione nulla di questo caso non potrebbe nemmeno essere paragonato alla variazione nulla di una università che appare più volte nel rank che però non ha modificato il reddito.

In [359]:
# uso la inner join sull'indice esplicito che setto come il nome dell'università.
# non posso farlo su quello precedente perché due università
# avrennero indici diversi dato che l'anno è diverso. 

ts_old.set_index('university_name', inplace = True)
ts_new.set_index('university_name', inplace = True)

mrg_2 = pd.merge(ts_old, ts_new, how = 'inner', left_index = True, right_index = True, suffixes = ['_old', '_new'])
mrg_2 = mrg_2[['income_new', 'income_old']]
mrg_2

,income_new,income_old
university_name,,
Tohoku University,74.7,82.3
École Polytechnique,82.3,-
Tokyo Institute of Technology,70.9,60.5
Trinity College Dublin,28.0,31.6
Tsinghua University,100.0,97.8
...,...,...
Lappeenranta University of Technology,80.3,73.8
University of Stuttgart,99.8,48.2
Scuola Normale Superiore di Pisa,52.0,40.1


In [360]:
# sembrerebbero non esserci valori nulli.
mrg_2.isnull().any()

income_new    False
income_old    False
dtype: bool

In [361]:
# eppure in Income vi sono valori mancanti segnati con -
# quindi butto via le righe che contengono -

mrg_2 = mrg_2[((mrg_2['income_new'].str.contains('\s*\-\s*')) | (mrg_2['income_old'].str.contains('\s*\-\s*'))) == False]

# Calcolo la differenza tra income.
mrg_2['DiffIncome'] = mrg_2['income_new'].astype('float')  - mrg_2['income_old'].astype('float')

# loc perché l'indice è esplicito (nome università).
mrg_2.loc[:, ['income_old', 'income_new', 'DiffIncome']].sort_values('university_name')

<ipython-input-361-d3a631ef3972>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrg_2['DiffIncome'] = mrg_2['income_new'].astype('float')  - mrg_2['income_old'].astype('float')


,income_old,income_new,DiffIncome
university_name,,,
Aalborg University,36.4,43.7,7.3
Aalto University,61.9,61.6,-0.3
Aarhus University,61.5,68.3,6.8
Aberystwyth University,35.5,31.3,-4.2
Alexandria University,36.0,29.7,-6.3
...,...,...,...
Yuan Ze University,29.2,39.8,10.6
Zhejiang University,70.3,96.2,25.9
École Normale Supérieure,30.7,37.1,6.4


### 3. Find the university with the largest increase computed in the previous point

In [362]:
mrg_2.DiffIncome.idxmax()

'TU Dresden'

### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

**_Strategia_**: prima di tutto, per i due rank rimasti, controllo se i nomi sono unici per ogni università. Successivamente,
prendo le università che si ripetono almeno due volte e ne estraggo le più recenti. Unisco le tre tabelle in corrispondenza del nome dell'università la quale avrà il rank mondiale per l'anno più recente per tutti e tre i rank. Alcuni valori di rank devono essere puliti: alcuni hanno = per indicare che delle università erano a pari merito nel rank ts ed alcuni valori sono definiti sotto forma di range.
Ho convertito tutti i valori in lista per poi concatenarli e scegliere il massimo e il minimo tra tutti i valori. In questo modo, ottengo la massima escursione di rank per tutte le università.

In [363]:
# Anche per cwur vedo se il nome è in corrispondenza biunivoca
# con l'università.
print('CWUR')
check_name(cwur)
print('SHANG')
# lui non ha l'info sulla nazione.
check_name(cwur, True, False)

CWUR
Nello stesso anno, sono presenti università con lo stesso nome?
No
Esistono università con lo stesso nome ma presenti in stati diversi?
No
SHANG
Nello stesso anno, sono presenti università con lo stesso nome?
No


In [364]:
# Per prima cosa costruisco la tabella per gli altri due rank delle università più recenti.
# Applico lo stesso metodo di prima, ovvero escludo le università che si presentano una sola volta nel rank.

print('CWUR')
cwur_old, cwur_new = select_old_and_new(cwur)
print('SHANGAY')
shang_old, shang_new = select_old_and_new(shang)


CWUR
Università singleton: 47, Università non singleton: 977
Dimensione df recenti (977, 5) e meno recenti (977, 5)
SHANGAY
Università singleton: 55, Università non singleton: 603
Dimensione df recenti (603, 4) e meno recenti (603, 4)


In [365]:
# non ci sono valori nulli da eliminare. 
ts_new['world_rank'].isnull().sum(), shang_new['world_rank'].isnull().sum(), cwur_new['world_rank'].isnull().sum()

(0, 0, 0)

In [366]:

# le dimensioni dei tre dataframe sono diversi: questo perché vi sono delle 
# nazioni considerate da un ranking che non sono state considerate dalle altre.
# Tra le tabelle devo fare una inner join perché
# per valutare il massimo e il minimo dei tre ranking è necessario conoscere
# tutti e 3. Se ci fosse un nan, non potrei dire nulla.

mrg_ts_new = ts_new[['world_rank']].reset_index()
mrg_shang_new = shang_new[['university_name','world_rank']]
mrg_cwur_new = cwur_new[['university_name','world_rank']]

appo_1 = pd.merge(mrg_ts_new, mrg_shang_new, on = 'university_name', how = 'inner', suffixes = ['_times', '_shang'])
mrg_ranking = pd.merge(appo_1, mrg_cwur_new, on = 'university_name', how = 'inner').sort_values('university_name')
mrg_ranking.columns = ['university_name', 'world_rank_times', 'world_rank_shang', 'world_rank_cwur']
mrg_ranking.head()

,university_name,world_rank_times,world_rank_shang,world_rank_cwur
20,Aalborg University,201-250,301-400,565
189,Aalto University,251-300,401-500,421
188,Aarhus University,=106,73,122
187,Arizona State University,189,93,97
186,Auburn University,501-600,401-500,549


In [367]:
mrg_ranking.isnull().any()

university_name     False
world_rank_times    False
world_rank_shang    False
world_rank_cwur     False
dtype: bool

In [368]:

#-- PULIZIA DEI TRE RANK.

# Prima di tutto, quando vi è =numero, è perché
# vi è un'altra università a pari merito in ts.
# Inoltre, gestiamo il fatto che alcuni hanno un range. 

# metto tutti i valori in una lista così da poterli concatenare
# e scegliere il massimo e il minimo.
def clan(rank): 
    if re.search('=', rank):
        return [int(re.sub('=', '', rank))]
    if re.search('-', rank):
        appo = re.search('^(\d+)-(\d+)', rank)
        return [int(appo.group(1)), int(appo.group(2))]
        
    return [int(rank)]
    
#capire se è meglio sintetizzare i valori nel rank oppure prendere il max excursus
mrg_ranking['world_rank_times'] = mrg_ranking['world_rank_times'].apply(clan)
mrg_ranking['world_rank_shang'] = mrg_ranking['world_rank_shang'].apply(clan)
# il terzo non ne ha bisogno perché non ci sono stringhe di quel tipo, sono tutti interi. 
mrg_ranking['world_rank_cwur'] = [[num]for num in mrg_ranking['world_rank_cwur']]

In [369]:
mrg_ranking.head()

,university_name,world_rank_times,world_rank_shang,world_rank_cwur
20,Aalborg University,"[201, 250]","[301, 400]",[565]
189,Aalto University,"[251, 300]","[401, 500]",[421]
188,Aarhus University,[106],[73],[122]
187,Arizona State University,[189],[93],[97]
186,Auburn University,"[501, 600]","[401, 500]",[549]


In [370]:

#-- CALCOLO MASSIMA ESCURSIONE DEL RANK PER OGNI UNIVERSITÀ.

def compute_max_rank(row):
    appo = row['world_rank_times'] + row['world_rank_shang'] + row['world_rank_cwur']
    return max(appo) - min(appo)

mrg_ranking['MaximumRanking'] = mrg_ranking.apply(compute_max_rank, axis = 1)
mrg_ranking[['university_name', 'MaximumRanking']]

,university_name,MaximumRanking
20,Aalborg University,364
189,Aalto University,249
188,Aarhus University,49
187,Arizona State University,96
186,Auburn University,199
...,...,...
104,Yale University,1
103,Yeshiva University,136
102,Yonsei University,252
101,York University,199


In [371]:
# check. 
mrg_ranking[mrg_ranking['university_name'] == 'Aarhus University']

,university_name,world_rank_times,world_rank_shang,world_rank_cwur,MaximumRanking
188,Aarhus University,[106],[73],[122],49


### 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

**_Strategia_**: considero solo le università che si presentano più di una volta per capire quella associata all'anno più recente. Elimino quelle che non presentano il numero di studenti nell'anno o la proporzione tra femmine e maschi. Calcolo il prodotto tra il numero di studenti totali nell'anno e la percentuale di donne per ricaverne il numero; in seguito ricavo quello degli uomini per differenza. Poiché considero un solo anno per università, non c'è la necessità di capire se effettivamente questi valori si modificano nel tempo. 

In [372]:
ts_new.reset_index(inplace = True)
ts_new_5 = ts_new[['university_name', 'female_male_ratio', 'num_students', 'country']]
ts_new_5.isnull().any()

university_name      False
female_male_ratio     True
num_students          True
country              False
dtype: bool

In [373]:
# li elimino perché altrimenti non posso calcolare il risultato.
ts_new_5 = ts_new_5[ts_new_5['num_students'].notnull() & ts_new_5['female_male_ratio'].notnull()]
ts_new_5.isnull().any()

university_name      False
female_male_ratio    False
num_students         False
country              False
dtype: bool

In [374]:
ts_new_5['Perc_Female'] = ts_new_5.female_male_ratio.str.extract(r'^\s*(\d+)\s*:\s*\d+').astype(float) / 100

ts_new_5['num_students'] = ts_new_5['num_students'].str.replace(',', '', regex=True).astype(int) 

In [376]:
ts_new_5['Num_Female'] = ts_new_5.apply(lambda row: int(row['num_students']*row['Perc_Female']), axis = 1)
ts_new_5['Num_Male'] = ts_new_5.apply(lambda row: row['num_students'] - row['Num_Female'], axis = 1)

In [377]:
sum_female_ts = ts_new_5.groupby('country')['Num_Female'].sum()
sum_male_ts = ts_new_5.groupby('country')['Num_Male'].sum()

In [378]:
pd.merge(sum_female_ts, sum_male_ts, left_index = True, right_index = True)

,Num_Female,Num_Male
country,,
Australia,327369,271842
Austria,66703,60232
Belgium,92916,76745
Brazil,52077,56420
Canada,278153,227933
Chile,2812,8906
China,127049,193600
Colombia,6940,8833
Czech Republic,31891,19547


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

**_Strategia_**: l'unico dataframe da considerare è ts perché è l'unico che conserva questa informazioni. Prima di tutto, è necessario capire se il rapporto tra femmine e maschi cambia nel tempo. Si osserva che non è così. Di conseguenza, posso eliminare l'anno ed eliminarei duplicati. Passiamo ora al calcolo del rapporto tra il numero di femmine e maschi. Si nota che vi è un'università di sole donne che fa scoppiare tale rapporto: l'ho eliminata per ottenere un valore più utile. Si osserva che in media ci sono più femmine che maschi. Prendiamo quindi quelle che hanno un rapporto minore, come richiesto.

In [48]:
# prima di tutto: quale dataset scegliere? ts perché è l'unico che contiene questa informazione.

# il rapporto cambia nel tempo?
check = ts[ts['female_male_ratio'].notnull()].groupby('university_name')['female_male_ratio'].nunique()
# rimane costante. 
check[check > 1]

Series([], Name: female_male_ratio, dtype: int64)

In [382]:
# l'indice esplicito di questa tabella è ciò che rimane dell'indice implicito di ts.

ts_6 = ts[['university_name', 'female_male_ratio']]
ts_6 = ts_6[ts_6['female_male_ratio'].notnull()].drop_duplicates()
ts_6 = ts_6[ts_6['university_name'] != 'Ewha Womans University']
ts_6.isnull().any()

university_name      False
female_male_ratio    False
dtype: bool

In [383]:
ts_6['FemaleGenderFloat'] = ts_6.female_male_ratio.str.extract(r'^\s*(\d+)\s*:\s*\d+').astype(float) 
ts_6['MaleGenderFloat'] = ts_6.female_male_ratio.str.extract(r'^\s*\d+\s*:\s*(\d+)').astype(float)

# Dal testo sembrerebbe che bisogna calcolare queasto rappporto: 
ts_6['Ratio'] = ts_6['FemaleGenderFloat']/ts_6['MaleGenderFloat']

# Calcolo la soglia che combacia con la media. 
thre = np.mean(ts_6['Ratio'])

# quindi in media, ci sono più donne che uomini nelle università
thre

1.081102092089725

In [384]:
unv_selected = ts_6[ts_6['Ratio'] < thre][['university_name']]
unv_selected = pd.DataFrame(unv_selected).sort_values('university_name')
unv_selected

,university_name
501,Aalborg University
502,Aalto University
476,Aberystwyth University
2406,Ajou University
146,Alexandria University
...,...
197,Zhejiang University
41,École Normale Supérieure
99,École Normale Supérieure de Lyon
38,École Polytechnique


### 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

**_Strategia_**: il primo aspetto da considerare è quello temporale. Poiché il numero di studenti rimane costante nel tempo per tutte le università di ts che riportano tale valore, possiamo prenederne uno qualsiasi. 
Steps:
1. Costruiamo una tabella da ts per calcolare il numero di studenti in ogni stato. 
2. Arricchiamo la lista delle università che abbiamo precedentemente calcolato con la nazione ed il numero di studenti in quelle università.
3. Aggreghiamo la tabella del punto 2 rispetto allo stato per capire quanti sono gli studenti che frequetano le università selezionate in quel determinato stato.
3. Per ogni stato, calcoliamo il rapporto tra gli studenti che frequentano le università selezionate in quello stato fratto il totale degli studenti che frequentano qualsiasi università in quello stato.

In [385]:
# la prima cosa da fare è calcolare tutti gli studenti in un certo stato.
df_7 = ts[['country', 'university_name', 'num_students']].drop_duplicates()
df_7.isnull().any()

country            False
university_name    False
num_students        True
dtype: bool

In [386]:
# elimino quelli che non hanno un numero di studenti.
df_7 = df_7[df_7['num_students'].notnull()]
df_7.isnull().any()

country            False
university_name    False
num_students       False
dtype: bool

In [387]:
# e gli altri li metto nel formato corretto.
df_7['num_students'] = df_7['num_students'].str.replace(',', '', regex=True).astype(int)
stud_per_cntry = df_7.groupby('country')['num_students'].sum()
stud_per_cntry

# IMPRTANTE: io sto prendendo il numero di studenti fissato un anno perché data una università,
# il numero di studenti si ripete negli anni nello stesso modo quindi la percentuale rimane la stessa.

country
Argentina                    108373
Australia                    743627
Austria                      134477
Bangladesh                    62716
Belarus                       29303
                             ...   
Uganda                        37340
Ukraine                       37096
United Arab Emirates          14447
United Kingdom              1324842
United States of America    3280303
Name: num_students, Length: 70, dtype: int32

In [388]:
# la seconda cosa da fare è selezionare da ts le università scelte, ovvero quelle 
# che hanno un numero di femmine rispetto ai maschi minore della media. 

# ottengo un dataframe con la nazione, l'università e il numero di strundetni.
# poiché questa informazione è ripetuta negli anni ma è sempre uguale, allora
# elimino le righe ridondanti.

# Adesso passo al filtraggio di df_7 con la lista delle università.

mrg_7 = pd.merge(unv_selected, df_7, on = 'university_name', how = 'left')
mrg_7 = mrg_7.groupby('country').sum()
mrg_7.reset_index(inplace = True)
mrg_7.columns = ['country', 'num_students_select']
mrg_7.head()

,country,num_students_select
0,Australia,146235
1,Austria,61033
2,Bangladesh,62716
3,Brazil,278774
4,Canada,86779


In [389]:
def ratio_country(row):
    return row['num_students_select']/stud_per_cntry[row['country']] * 100

mrg_7['Perc'] = mrg_7.apply(ratio_country, axis = 1)
mrg_7.head()

,country,num_students_select,Perc
0,Australia,146235,19.665101
1,Austria,61033,45.385456
2,Bangladesh,62716,100.000000
3,Brazil,278774,52.137695
4,Canada,86779,12.102157


In [391]:
# Si osserva che vi sono delle nazioni con frazione uguale ad uno.
# Questo perché quelle nella lista sono le uniche nel loro stato entrate nel rank.
mrg_7[mrg_7['Perc'] == max(mrg_7['Perc'])]

,country,num_students_select,Perc
2,Bangladesh,62716,100.0
7,Colombia,54645,100.0
15,Ghana,37274,100.0
20,Indonesia,43565,100.0
25,Kenya,72676,100.0
26,Lebanon,7695,100.0
27,Luxembourg,5144,100.0
29,Mexico,151731,100.0
30,Morocco,56060,100.0
33,Nigeria,26247,100.0


### 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [392]:
att = pd.read_csv('Dataset/educational_attainment_supplementary_data.csv')
att.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [393]:
att.dropna(subset=['country_name', 'series_name'], axis = 0, inplace = True, how = 'any')

In [394]:
att.isnull().any()

country_name    False
series_name     False
1985             True
1986             True
1987             True
1990             True
1991             True
1992             True
1993             True
1995             True
1996             True
1997             True
1998             True
1999             True
2000             True
2001             True
2002             True
2003             True
2004             True
2005             True
2006             True
2007             True
2008             True
2009             True
2010             True
2011             True
2012             True
2013             True
2015             True
dtype: bool

###  9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [395]:
att.columns, att.shape

(Index(['country_name', 'series_name', '1985', '1986', '1987', '1990', '1991',
        '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
        '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
        '2011', '2012', '2013', '2015'],
       dtype='object'),
 (79050, 29))

In [396]:
%%time 

mlt = pd.melt(att, id_vars=['country_name', 'series_name'], value_vars=['1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015'])

mlt = mlt.rename({'variable':'year'})
mlt.head()

Wall time: 1.28 s


,country_name,series_name,variable,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54


In [397]:
mlt.shape

(2134350, 4)

In [398]:
79050*27

2134350

### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking

**_Strategia_**: prendo un dataframe con una sola colonna con tutte le università che appaiono nei tre ranking per tutti gli anni. In seguito, effettuo tre left join tra il df precedente ed i tre dataframe che sono stati forniti dai quali seleziono solo solo la lista delle università. In questo modo, in ognuna delle left join, se una università di un ranking si trova nella lista totale, allora verrà agganciata a tale valore, altriementi Nan. Per concludere, vado a vedere nel dataframe finale (senza la lista) chi ha valore nullo, sommo sulle righe ed ottengo in quanti ranking appare ogni università.

In [51]:
# Facciamo una left join per ognuna.
un_ts = pd.DataFrame(set(ts[ts.university_name.notnull()]['university_name']), columns = ['university_name'])
un_sh = pd.DataFrame(set(shang[shang.university_name.notnull()]['university_name']), columns = ['university_name'])
un_cw = pd.DataFrame(set(cwur[cwur.university_name.notnull()]['university_name']), columns = ['university_name'])

# Questo df contiene tutte le possibili università.
list_un = pd.DataFrame(set(pd.concat([un_ts, un_sh, un_cw])['university_name']), columns = ['University'])
list_un

,University
0,"Queen Mary, U. of London"
1,University of Udine
2,Oregon State University
3,University of Oklahoma - Norman Campus
4,Adam Mickiewicz University in Poznań
...,...
1447,University of the Mediterranean (Aix-Marseille 2)
1448,École Polytechnique Fédérale de Lausanne
1449,National University of Córdoba
1450,Federal University of Santa Catarina


In [52]:
# ci sono delle università in comune. 
list_un.shape[0] == un_ts.shape[0] + un_sh.shape[0] + un_cw.shape[0]

False

In [70]:
# per la left join.
un_ts.rename({'university_name': 'university_name_ts'}, axis = 1, inplace = True)
un_sh.rename({'university_name': 'university_name_sh'}, axis = 1, inplace = True)
un_cw.rename({'university_name': 'university_name_cw'}, axis = 1, inplace = True)

In [79]:
appo = pd.merge(list_un, un_ts, left_on = 'University', right_on='university_name_ts', how = 'left')
appo_1 = pd.merge(appo, un_sh, left_on = 'University', right_on='university_name_sh', how = 'left')
rank_count = pd.merge(appo_1, un_cw, left_on = 'University', right_on='university_name_cw', how = 'left')
#names= ["_ts","_sh","_cw"]
#dfs = [list_un,un_ts,un_sh,un_cw]
#rank_count = dfs[0].join(dfs[1:],how="left",rsuffix=names[0:],lsuffix="University") 
rank_count['RankTotal'] = rank_count.iloc[:,1:4].notnull().apply(sum, axis = 1)

In [80]:
rank_count

,University,university_name_ts,university_name_sh,university_name_cw,RankTotal
0,"Queen Mary, U. of London",NaN,"Queen Mary, U. of London",NaN,1
1,University of Udine,NaN,NaN,University of Udine,1
2,Oregon State University,Oregon State University,Oregon State University,Oregon State University,3
3,University of Oklahoma - Norman Campus,NaN,NaN,University of Oklahoma - Norman Campus,1
4,Adam Mickiewicz University in Poznań,NaN,NaN,Adam Mickiewicz University in Poznań,1
...,...,...,...,...,...
1447,University of the Mediterranean (Aix-Marseille 2),NaN,University of the Mediterranean (Aix-Marseille 2),NaN,1
1448,École Polytechnique Fédérale de Lausanne,École Polytechnique Fédérale de Lausanne,NaN,NaN,1
1449,National University of Córdoba,National University of Córdoba,NaN,National University of Córdoba,2
1450,Federal University of Santa Catarina,Federal University of Santa Catarina,NaN,Federal University of Santa Catarina,2


In [62]:
rank_count[['University', 'RankTotal']].sort_values('University')

,University,RankTotal
91,AGH University of Science and Technology,3
42,Aalborg University,3
190,Aalto University,3
224,Aarhus University,3
831,Aberystwyth University,1
...,...,...
778,École centrale de Lyon,2
945,École normale supérieure - Paris,1
1261,École normale supérieure de Cachan,0
562,École normale supérieure de Lyon,3


In [148]:
len(set(rank_count.University))

1452

### 11. In the times ranking, compute the number of times each university appears

In [70]:
ts['university_name'].isnull().any()

False

In [71]:
grp = ts.groupby('university_name').size()
grp = pd.DataFrame(grp, columns = ['Appearing']).reset_index()
grp

,university_name,Appearing
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


### 12. Find the universities that appear at most twice in the times ranking

In [72]:
grp[grp['Appearing'] < 3]

,university_name,Appearing
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

**_Strategia_**: prima di tutto, dalla tabella precedentemente ottenuta seleziono le università che sono presenti almeno per un anno in tutti e tre i rank. Creo tre tabelle che derivano dall'arricchimento della tabella precedente con rank e anni di riferimento per ognuno dei tre rank prendo solo gli anni in comune tra tutte e tre le tabelle. Concateno le tre tabelle ed infine, le srotolo rispetto agli anni per poter fare una groupby sull'anno e sull'università avendo a disposizione la differenza dei tre rank. Qui però c'è un problema: dato che alcuni rank sono sotto forma di range, cosa si intende con la "stessa posizione" nei tre rank? Secondo me non può essere definita in questo caso e li ho eliminati. Il risultato è che per nessuna università si verifica questa situazione. 

Per concludere, si potrebbe però implementare una soluzione in cui si può renedere più blando il concetto di "stessa posizione" includendo anche una possibile sovrapposizione dei range.

In [150]:
# Mi sta chiedendo di trovare le università che almeno una volta
# hanno ottenuto la stessa posizione in tutti e tre i rank.

In [169]:
# prima di tutto, prendo le università che hanno un valore in tutti e tre i rank. 
# potrebbero però essere presenti in anni differenti.
three_rank = rank_count[rank_count['RankTotal'] == 3]
three_rank = three_rank[['University']]
three_rank.rename({'University': 'university_name'}, inplace = True, axis = 1)
three_rank

,university_name
3,Queen Mary University of London
11,Temple University
18,Chalmers University of Technology
22,University of Fribourg
23,University of Cape Town
...,...
1430,University of Naples Federico II
1433,Technical University of Berlin
1442,Boston College
1447,Hanyang University


In [170]:
len(set(three_rank.university_name))

362

In [171]:
# ts_y = ts[~ts.world_rank.astype(str).str.contains(r'-')]
ts_y = ts
ts_y = ts_y[(ts_y['year'] >= 2012) & (ts_y['year'] <= 2015)]
#
ts_y = ts_y[['university_name', 'year', 'world_rank']].pivot(index='university_name', columns='year', values='world_rank')
ts_y_sel = pd.merge(three_rank, ts_y, left_on = 'university_name', right_index = True, how = 'left')
ts_y_sel['Type'] = np.repeat('ts', ts_y_sel.shape[0])
ts_y_sel.head()

,university_name,2012,2013,2014,2015,Type
3,Queen Mary University of London,127,145,114,107,ts
11,Temple University,NaN,351-400,351-400,NaN,ts
18,Chalmers University of Technology,226-250,226-250,276-300,276-300,ts
22,University of Fribourg,NaN,301-350,276-300,276-300,ts
23,University of Cape Town,103,113,126,124,ts


In [172]:
cwur_y = cwur[['university_name', 'year', 'world_rank_cwur']].pivot(index = 'university_name', columns='year', values='world_rank_cwur')
cwur_y_sel = pd.merge(three_rank, cwur_y, left_on = 'university_name', right_index = True, how = 'left')
cwur_y_sel['Type'] = np.repeat('cwur', cwur_y_sel.shape[0])
cwur_y_sel.head()

,university_name,2012,2013,2014,2015,Type
3,Queen Mary University of London,NaN,NaN,214.0,209.0,cwur
11,Temple University,NaN,NaN,390.0,381.0,cwur
18,Chalmers University of Technology,NaN,NaN,395.0,402.0,cwur
22,University of Fribourg,NaN,NaN,514.0,573.0,cwur
23,University of Cape Town,NaN,NaN,267.0,248.0,cwur


In [173]:
# shang_y = shang[~shang.world_rank.astype(str).str.contains(r'-')]
shang_y = shang
shang_y = shang_y[['university_name', 'year', 'world_rank']]
shang_y = shang_y[(shang_y['year'] >= 2012) & (shang_y['year'] <= 2015)].drop_duplicates()
shang_y = shang_y.pivot(index = 'university_name', columns='year', values='world_rank')
shang_y_sel = pd.merge(three_rank, shang_y, left_on = 'university_name', right_index = True, how = 'left')
shang_y_sel['Type'] = np.repeat('shang', shang_y_sel.shape[0])
shang_y_sel.head()

,university_name,2012,2013,2014,2015,Type
3,Queen Mary University of London,NaN,NaN,NaN,151-200,shang
11,Temple University,NaN,NaN,301-400,301-400,shang
18,Chalmers University of Technology,NaN,NaN,301-400,301-400,shang
22,University of Fribourg,NaN,NaN,NaN,NaN,shang
23,University of Cape Town,201-300,NaN,201-300,201-300,shang


In [399]:
three_concat = pd.concat([ts_y_sel, cwur_y_sel, shang_y_sel])

three_concat[three_concat['university_name'] == 'Chalmers University of Technology']
three_concat.head()

,university_name,2012,2013,2014,2015,Type
3,Queen Mary University of London,127,145,114,107,ts
11,Temple University,NaN,351-400,351-400,NaN,ts
18,Chalmers University of Technology,226-250,226-250,276-300,276-300,ts
22,University of Fribourg,NaN,301-350,276-300,276-300,ts
23,University of Cape Town,103,113,126,124,ts


In [175]:
three_concat = three_concat.melt(id_vars=['university_name', 'Type'], value_vars=[2012, 2013, 2014, 2015])
three_concat = three_concat[three_concat['value'].notnull()]
three_concat.head()

,university_name,Type,variable,value
0,Queen Mary University of London,ts,2012,127
2,Chalmers University of Technology,ts,2012,226-250
4,University of Cape Town,ts,2012,103
6,Nanjing University,ts,2012,251-275
7,McMaster University,ts,2012,65


In [176]:
def check_three_equal(row):
    # ogni valore è una serie.
    if row.shape[0] == 3:
        if row.iloc[0,] == row.iloc[1,] == row.iloc[2,]:
            return True
        
    return False
        

appo= three_concat.groupby(['variable','university_name'])['value'].apply(check_three_equal)
sum(appo.reset_index()['value'])

0

In [81]:
# Si potrebbe anche fare con la sovrapposizione dei range.
# cwur[cwur["university_name"]=="Queen Mary University of London"]

,university_name,country,world_rank,year
413,Queen Mary University of London,United Kingdom,214,2014
1408,Queen Mary University of London,United Kingdom,209,2015
